In [17]:
import json
import pandas as pd
import sqlalchemy
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pytz


In [18]:
# Get keys from my_secrets_key.py file
from my_secrets_key import schema_name,host_name,user_name,db_password,port,weather_api_key,aerodatabox_api_key


In [19]:
schema = schema_name
host = host_name
user = user_name
password = db_password
port = port
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [20]:
UNITS = 'metric'  # You can change this to 'imperial' or 'standard' if needed
BASE_URL = 'http://api.openweathermap.org/data/2.5/forecast'
API_KEY = weather_api_key


Get Weather Data

In [23]:
def current_time():  
    current_date = None
    # Define the timezone
    tz = pytz.timezone('Europe/Berlin')
    # Get the current time in the 'Europe/Berlin' timezone
    current_time = datetime.now(tz)
    # Extract the date from the current time
    current_date = current_time.strftime("%Y-%m-%d %H:%M:%S")
   
    return current_date

def get_weather_data_for_city(city_name,city_id):
    city_url = f"{BASE_URL}?q={city_name}&appid={API_KEY}&units={UNITS}"
    city_response = requests.get(city_url)
    if city_response.status_code == 200:
        weather_data = []
        city_data = city_response.json()
        forecast = city_data.get('list', [])
        city = city_data['city']['name']
        country = city_data['city']['country']
        sunset = datetime.fromtimestamp(city_data["city"]["sunset"]).strftime('%H:%M:%S')
        
        for item in forecast:
            time =  pd.to_datetime(item['dt_txt'])
            temperature = item['main']['temp']
            whether_outlook = item['weather'][0]['main']
            description = item['weather'][0]['description']
            feels_like_temperature = item['main']['feels_like']
            humidity = item['main']['humidity']
            
            try:
                snow = item['snow']['3h']
            except KeyError:
                snow = 0

            try:
                rain = item['rain']['3h']
            except KeyError:
                rain = 0
                
            pressure = item['main']['pressure']
            wind_speed = item['wind']['speed']
            prob_prec = item['pop']
            clouds = item['clouds']['all']

            info_retrieved_at =  current_time()
            
            weather_data.append({
                'city_id': city_id,
                'country_code': country,
                'forecasttime': time,
                'Weather_outlook' : whether_outlook,
                'Weather_description': description,
                'temperature': temperature,
                'temperature_feels_like': feels_like_temperature,
                'humidity': humidity,
                'snow': snow,
                'rain': rain,
                'wind_speed': wind_speed,
                'prob_prec': prob_prec,
                'info_retrieved_at': info_retrieved_at,

            })
        
        return weather_data

def get_weather_data_for_cities(city_df):
    
    all_city_data = []
    for i, city_id in enumerate(city_df['city_id']):
        city_name = city_df.iloc[i]['city_name']
        city_id = city_df.iloc[i]['city_id']
        weather_data = get_weather_data_for_city(city_name,city_id)
        weather_data
        if weather_data is not None:
          all_city_data.extend(weather_data)
    if all_city_data:
        return  pd.DataFrame(all_city_data)
    else:
        print("No weather data retrieved for any city.")
        return None

       
city_df = pd.read_sql('cities',con=con)
new_weather_df = get_weather_data_for_cities(city_df)
new_weather_df

,city_id,country_code,forecasttime,Weather_outlook,Weather_description,temperature,temperature_feels_like,humidity,snow,rain,wind_speed,prob_prec,info_retrieved_at
0,25,DE,2023-11-12 21:00:00,Clouds,scattered clouds,6.61,4.71,81,0,0.00,2.62,0.03,2023-11-12 19:10:53
1,25,DE,2023-11-13 00:00:00,Clouds,scattered clouds,5.38,3.35,78,0,0.00,2.50,0.00,2023-11-12 19:10:53
2,25,DE,2023-11-13 03:00:00,Clouds,few clouds,3.86,1.77,78,0,0.00,2.26,0.00,2023-11-12 19:10:53
3,25,DE,2023-11-13 06:00:00,Clouds,scattered clouds,3.48,1.68,79,0,0.00,1.93,0.00,2023-11-12 19:10:53
4,25,DE,2023-11-13 09:00:00,Clouds,broken clouds,5.83,3.79,70,0,0.00,2.62,0.00,2023-11-12 19:10:53
...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,32,IN,2023-11-17 06:00:00,Clouds,overcast clouds,29.69,32.63,62,0,0.00,2.03,0.00,2023-11-12 19:10:55
316,32,IN,2023-11-17 09:00:00,Clouds,overcast clouds,31.58,33.83,51,0,0.00,1.51,0.04,2023-11-12 19:10:55
317,32,IN,2023-11-17 12:00:00,Clouds,overcast clouds,28.98,31.47,63,0,0.00,1.80,0.08,2023-11-12 19:10:55
318,32,IN,2023-11-17 15:00:00,Clouds,few clouds,26.53,26.53,71,0,0.00,1.72,0.01,2023-11-12 19:10:55


In [24]:
new_weather_df.to_sql(name='weathers', con=con, if_exists='append', index=False)


320